# Three methods for decoding run length encoding (RLE) in TensorFlow

Run length encoding is a method to store segmentations in an efficient manner and is a frequently used data format in Kaggle competitions involving segmentation. You can read more about it [here](https://www.kaggle.com/lifa08/run-length-encode-and-decode).



In [134]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import time
import sys
%matplotlib inline

## Data
From [Kaggle Caravana Image Masking challenge](https://www.kaggle.com/c/carvana-image-masking-challenge)


In [135]:
df = pd.read_csv('/Users/user/Downloads/train_masks.csv')
masks = df.rle_mask.values
shape = (1280,1918)

## Numpy

In [136]:
def rle_decode_np(mask_rle, shape):
    '''
    Source: https://www.kaggle.com/lifa08/run-length-encode-and-decode)
    
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

## TensorFlow

### Method 1 
Somewhat wasteful approach that generates ranges for each (start, length) pair of size max(lengths), adds them to the starts and selects only the indices for each pair, that are less the end value for that pair.

In [137]:
def rle_decode_tf0(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns tf tensor, 1 - mask, 0 - background

    '''
    s = tf.cast(tf.strings.to_number(tf.strings.split(tf.constant([mask_rle]), ' ').values), tf.int32)
    starts = s[0::2]
    lengths = s[1::2]
    starts -= 1
    ends = starts + lengths
    max_range = tf.range(tf.reduce_max(lengths))
    ranges = starts[:, None] + max_range[None]
    ranges_mask = tf.less(ranges, ends[:, None])
    inds = tf.boolean_mask(ranges, ranges_mask)
    img = tf.scatter_nd(inds[:, None], tf.ones_like(inds), [shape[0]*shape[1]])
    return tf.cast(tf.reshape(img, shape), tf.uint8)

### Method 2 
Somewhat complicated approach that first takes a cumulative sum over all the lengths and then arranges them in a vector so that which subtract from a range of size sum(lengths) the range gets restarted for each (start, length) pair. The starts are all also arranged similarly to the lengths so that when added to these new ranges, they give the mask indices. 

In [138]:
def rle_decode_tf1(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns tf tensor, 1 - mask, 0 - background

    '''
    s = tf.cast(tf.strings.to_number(tf.strings.split(tf.constant([mask_rle]), ' ').values), tf.int32)
    starts = s[0::2]
    lengths = s[1::2]
    starts -= 1
    ends = starts + lengths
    num_inds = tf.reduce_sum(lengths)
    lengths_cumsum = tf.cumsum(lengths, exclusive=True)
    starts_indicator = tf.scatter_nd(lengths_cumsum[:, None], tf.ones_like(lengths_cumsum), [num_inds])
    row_inds = tf.cumsum(starts_indicator) - 1
    lengths_shift = tf.gather(lengths_cumsum, row_inds)
    starts_repeat = tf.gather(starts, row_inds)
    inds = starts_repeat + tf.range(num_inds) - lengths_shift
    img = tf.scatter_nd(inds[:, None], tf.ones_like(inds), [shape[0]*shape[1]])
    return tf.cast(tf.reshape(img, shape), tf.uint8)

### Method 3
A clean, simple approach made possible by the tf.ragged functionality in TensorFlow 2.0 which makes it possible to generate a set of variable size ranges for pairs of starts and lengths which is exactly what we require.

In [139]:
def rle_decode_tf2(mask_rle, shape):
    '''
    Requires TF 2.0
    
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns tf tensor, 1 - mask, 0 - background

    '''
    s = tf.cast(tf.strings.to_number(tf.strings.split(tf.constant([mask_rle]), ' ').values), tf.int32)
    starts = s[0::2]
    lengths = s[1::2]
    starts -= 1
    ends = starts + lengths
    inds = tf.ragged.range(starts, ends).values
    img = tf.scatter_nd(inds[:, None], tf.ones_like(inds), [shape[0]*shape[1]])
    return tf.cast(tf.reshape(img, shape), tf.uint8)

In [130]:
time_np = 0
time_tf = [0, 0, 0]
for i, mask_rle in enumerate(masks, 1):
    t = time.time()
    mask_np = rle_decode_np(mask_rle, shape)
    time_np += (time.time() - t)
    time_str = 'np: {:.4f}'.format(time_np)
    for j in range(3):
        rle_decode_tf = globals()['rle_decode_tf{}'.format(j)]

        t = time.time()
        mask_tf = rle_decode_tf(mask_rle, shape).numpy()
        time_tf[j] += (time.time() - t)
        

        assert(np.all(mask_np == mask_tf))
        
    time_str += (', ' + ', '.join(map('tf{}: {:.4f}'.format, range(3), time_tf)))

    sys.stdout.write('\r{}/{}, {}'.format(i, len(masks), time_str))

5088/5088, np: 12.3134, tf0: 286.3517, tf1: 301.1048, tf2: 208.5881

Notably all the TensorFlow versions are considerably slower than the numpy version. Both the versions without tf.ragged are similar in speed whilst the one with it is faster although still much slower than the numpy one. I have yet to investigate the reason for this and in particular whether they would still be as slow on a GPU. 